# Колодий Антон, гр. 8376

Я взял новый датасет с "объявлениями" о продаже автомобилей. 
Очень долго пытался объединять данные так, чтобы получались хорошие кластеры, но прям явных групп у меня так и не получилось.
Но тем не менее:

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('cars.csv')
#В датасете есть лишний столбец индексов, уберем его сразу
df = df.drop(df.columns[[0]], axis=1)
df.head()

В датасете представлены марки и модели автомобилей, продающихся в какой-то стране. Так же есть информация о двигателе, годе выпуска, о пробеге, типе топлива и коробке передач.


Введем новый атрибут возраст авто: age

# AGE

In [ ]:
interpret = pd.Series([])
for i in range(len(df)):
    interpret[i]=2023 - df["year"][i]

In [ ]:
df.insert(5,"age", interpret)

# Результат введения нового атрибута

In [ ]:
df

В датасете не оказалось выбросов, поэтому я стер за ненадобностью применение метода z-оценки. Точнее были выброшены некоторые дорогие автомобили, но это не выбросы, они действительно штучно продаются.

# Графики зависимости от введенного атрибута

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.scatter(df["age"], df["brand"], c="red", s=20)
plt.xlabel("age")
plt.xticks(range(1, 20))
plt.ylabel("brand")

In [ ]:
plt.scatter(df["age"], df["price (eur)"], c="green")
plt.xlabel("age")
plt.xticks(range(1, 20))
plt.ylabel("price")

# График зависимости стоимости от марки (или наоборот)

In [ ]:
plt.scatter(df["price (eur)"], df["brand"], c="blue")
plt.ylabel("brand")
plt.xlabel("price")

# Промежуточный итог

К сожалению, такой подход не позволяет нормально определить кластеры, кроме как "Основная масса сосредоточена в цене от 10 до 20 тыс." Связано это с тем, что представлено много брендов и моделей и все они перемешаны.

Попробуем хотя бы рассматривать зависимости в рамках определенных стран-производителей. Но я не буду брать все авто-производители страны, чтобы не было вновь значительного наложения данных.

In [ ]:
#Германия
deutch_df = df.loc[(df['brand'].isin(['Mercedes', 'BMW', 'Volkswagen']))]
plt.scatter(deutch_df["price (eur)"], deutch_df["brand"], c="orange")
plt.ylabel("brand")
plt.xlabel("price")

In [ ]:
plt.scatter(deutch_df["age"], deutch_df["price (eur)"], c="green")
plt.xlabel("age")
plt.xticks(range(1, 20))
plt.ylabel("price")

Теперь мы видим небольшой кластер дорогих(>50тыс) автомобилей возрастом 5-6 лет, большой кластер автомобилей возрастом 5-8 лет стоимостью 20-40 тысяч и небольшой кластер автомобилей со стоимостью около 10 тысяч. 


In [ ]:
#Япония
jap_df = df.loc[(df['brand'].isin(['Nissan', 'Mitsubishi', 'Honda']))]
plt.scatter(jap_df["price (eur)"], jap_df["brand"], c="orange")
plt.ylabel("brand")
plt.xlabel("price")

In [ ]:
plt.scatter(jap_df["age"], jap_df["price (eur)"], c="green")
plt.xlabel("age")
plt.xticks(range(1, 20))
plt.ylabel("price")

Здесь тоже заметно несколько кластеров: машины возрастом от 10 до 12 лет стоимостью около 10 тыс. формируют один кластер.
Машины возрастом 5-8 лет со стоимостью 15-20 тысяч второй явный кластер.
Машины дешевле 15 тысяч и возрастом 4-7 лет - еще один кластер.


In [ ]:
#Франция
fr_df = df.loc[(df['brand'].isin(['Renault', 'Citroen', 'Peugeot']))]
plt.scatter(fr_df["price (eur)"], fr_df["brand"], c="orange")
plt.ylabel("brand")
plt.xlabel("price")

In [ ]:
plt.scatter(fr_df["age"], fr_df["price (eur)"], c="green")
plt.xlabel("age")
plt.xticks(range(1, 20))
plt.ylabel("price")

Здесь тоже можно выделить несколько кластеров: автомобили с ценой выше 15 тысяч, 
автомобили возрастом 2-11 лет с ценой ниже 15 тысяч
и 3 старых автомобиля 13-19 лет.

Но явных групп так  ине удалось выявить. Попробуем объединить предыдущие три страны, может что-то получится. И заменим bmw на opel.

In [ ]:
group_df = df.loc[(df['brand'].isin(['Mercedes', 'Opel', 'Volkswagen','Nissan', 'Mitsubishi', 'Honda','Renault', 'Citroen', 'Peugeot']))]
plt.scatter(group_df["price (eur)"], group_df["brand"], c="orange")
plt.ylabel("brand")
plt.xlabel("price")

In [ ]:
plt.scatter(group_df["age"], group_df["price (eur)"], c="green")
plt.xlabel("age")
plt.xticks(range(1, 20))
plt.ylabel("price")

Давайте попробуем взять 2 самые дешевые и 2 дорогие марки в рамках датасета

In [ ]:
#2 дешевые и 2 дорогие
vs_df = df.loc[(df['brand'].isin(['Renault', 'Mini', 'Mercedes', 'Land']))]
plt.scatter(vs_df["price (eur)"], vs_df["brand"], c="orange")
plt.ylabel("brand")
plt.xlabel("price")

In [ ]:
plt.scatter(vs_df["age"], vs_df["price (eur)"], c="green")
plt.xlabel("age")
plt.xticks(range(1, 20))
plt.ylabel("price")

Здесь можно выделить 4 кластера. 3 точки наверху, затем в цене около 30-40 тысяч, затем две группы примерно в разрезе 20 тысяч.

# K-Means

По двум рассмотренным группам предлагаю вариант применения метода KMeans.

In [ ]:
#начнем с группы двух дорогих и 2 недорогих брендов
vs_df

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
cluster_df = pd.concat([vs_df["age"], vs_df["price (eur)"]], axis=1, keys=["age", "price (eur)"])
kmeans=KMeans(n_clusters=4)
kmeans.fit(cluster_df)
label = kmeans.fit_predict(cluster_df)

In [ ]:
#Векторы центров кластеров
kmeans.cluster_centers_

In [ ]:
#Построим графики
u_labels = np.unique(label)
 
for i in u_labels:
    plt.scatter(cluster_df.iloc[label == i , 0] , cluster_df.iloc[label == i , 1] , label = i)
plt.legend()
plt.xlabel("age")
plt.ylabel("price")
plt.show()

Метод K-средних очень хорошо представил кластеризацию, как я описывал ранее.

Теперь посмотрим на группу немецких автомобилей, там тоже можно было выделить кластеры.

In [ ]:
cluster_df1 = pd.concat([deutch_df["age"], deutch_df["price (eur)"]], axis=1, keys=["age", "price (eur)"])
kmeans=KMeans(n_clusters=3)
kmeans.fit(cluster_df1)
label = kmeans.fit_predict(cluster_df1)

In [ ]:
#Построим график
u_labels = np.unique(label)
 
for i in u_labels:
    plt.scatter(cluster_df1.iloc[label == i , 0] , cluster_df1.iloc[label == i , 1] , label = i)
plt.legend()
plt.xlabel("age")
plt.ylabel("price")
plt.show()

Метод K-средних определил третьим кластером автомобили дешевле 20 000, когда я предсказывал около 10 000. Но остальное правильно.